This file is for testing functions in des helpers.

In [74]:
import pandas as pd
from des_helpers import select_function, dpa_select_function, encrypt, decrypt, bin2hex, des, preprocess_key, dec2bin, bin2dec

# Load data
trace_df = pd.read_csv("des_extension_data.csv")

ciphertexts = [c for c in trace_df['Ciphertext']]
plaintexts = [p for p in trace_df['Plaintext']]

In [75]:
trace_df.iloc[0]

Key           000000000000003C
Plaintext     959031148A37CAEE
Ciphertext    6ED1AA24370D4CA2
Power                     1208
Name: 0, dtype: object

Extracting Key for of first sbox in first round this will be what we will get from our analysis:

In [76]:
sbox_key = bin2dec(int(preprocess_key(f"{60:016x}".upper())[0][:6]))
print(f"Key for first sbox in first round: {sbox_key}")

Key for first sbox in first round: 4


In [77]:
# Test encrypt decrypt
sample = trace_df.iloc[0]
key = f"{60:016x}".upper()
pt = sample["Plaintext"].upper()
ct = sample["Ciphertext"].upper()

text = bin2hex(encrypt(pt, key))
print(text == ct)

text = bin2hex(decrypt(ct, key))
print(text == pt)

True
True


Here the sbox output we are interested in is the first sbox in the first round of encryption. This should be the same output at the first sbox in the last round.

In [78]:
# Test select func
def mod_decrypt(ciphertext, key):
	rkb = preprocess_key(key)
	rkb_rev = rkb[::-1]
	text, sbox_out = des(ciphertext, rkb_rev)

	return text, sbox_out

def mod_encrypt(plaintext, key):
	rkb = preprocess_key(key)
	text, sbox_out = des(plaintext, rkb)

	return text, sbox_out

for i in range(10):
	ct = ciphertexts[i]
	pt = plaintexts[i]

	text, sbox_out = mod_encrypt(pt, key)
	print(f"Encryption S-Box Output for first round first s-box: {dec2bin(sbox_out[0][0])}")

	text, sbox_out = mod_decrypt(ct, key)
	print(f"Decryption S-Box Output for last round first s-box: {dec2bin(sbox_out[15][0])}")

Encryption S-Box Output for first round first s-box: 0111
Decryption S-Box Output for last round first s-box: 0111
Encryption S-Box Output for first round first s-box: 0010
Decryption S-Box Output for last round first s-box: 0010
Encryption S-Box Output for first round first s-box: 0110
Decryption S-Box Output for last round first s-box: 0110
Encryption S-Box Output for first round first s-box: 0000
Decryption S-Box Output for last round first s-box: 0000
Encryption S-Box Output for first round first s-box: 0100
Decryption S-Box Output for last round first s-box: 0100
Encryption S-Box Output for first round first s-box: 0010
Decryption S-Box Output for last round first s-box: 0010
Encryption S-Box Output for first round first s-box: 1101
Decryption S-Box Output for last round first s-box: 1101
Encryption S-Box Output for first round first s-box: 1001
Decryption S-Box Output for last round first s-box: 1001
Encryption S-Box Output for first round first s-box: 0001
Decryption S-Box Outpu

Here we check the outputs of the select function under a bunch of keys ideally we are looking for a fairly even distribution of outputs

In [79]:
import numpy as np

for key in range(0,64):
	out = []

	for pt in plaintexts:
		sbox_out = select_function(pt, key)
		out.append({"pt": pt, "s-box out": sbox_out})

	print(f"S-box outs for key = {key}")
	print(pd.DataFrame(out)["s-box out"].value_counts())

S-box outs for key = 0
s-box out
0001    277
1101    273
0110    272
0100    269
0101    267
1001    260
0000    260
1011    259
0111    257
1110    255
0011    250
0010    248
1111    247
1100    240
1000    236
1010    226
Name: count, dtype: int64
S-box outs for key = 1
s-box out
1101    283
0100    273
1111    272
1100    272
0011    272
0000    261
1010    261
1000    255
0110    255
1110    252
1011    250
0010    245
0111    238
1001    238
0001    237
0101    232
Name: count, dtype: int64
S-box outs for key = 2
s-box out
0100    280
0111    277
0000    271
1001    263
1101    262
1010    261
1000    258
0010    254
1110    253
0110    252
0101    250
0001    249
1111    248
1011    243
1100    239
0011    236
Name: count, dtype: int64
S-box outs for key = 3
s-box out
1110    283
1111    277
1000    274
0110    268
1011    266
1001    261
0000    259
0001    258
0011    257
1101    256
0101    250
0111    241
0010    240
0100    240
1100    240
1010    226
Name: count, dtype: in

Here we verify that the select function outputs the correct value by comparing it to the output of a normal encrypt

In [80]:
# Test on entire dataset
results = []
key = f"{60:016x}".upper()
for pt in plaintexts:
	text, sbox_out = mod_encrypt(pt, key)
	encrypt_out = int(dec2bin(sbox_out[0][0])[0])
	select_out = dpa_select_function(pt, 4) # correct key is 4
	
	if encrypt_out == select_out: 
		results.append(True)
	else:
		print(encrypt_out, select_out)
		results.append(False)
		
print(all(results))

True
